# Preprocessing image for yolo training

In [ ]:
import shutil # copy, move file
import os # miscellaneous operation system interfaces
import pathlib
import pandas as pd
from tqdm import tqdm


In [2]:
!pwd

/home/jupyter/food-detection/Dataset


In [10]:
obj_path = 'obj'
if not os.path.isdir(obj_path):
    os.mkdir(obj_path)

# Download image from OpenImage

Enter a script below to terminal in same folder of this file.

``python openimages/src/openimages/download.py --csv_dir csv_dir --base_dir images --format darknet --labels Apple Beer Cucumber Radish Pancake Waffle Bagel Popcorn Burrito Cheese Muffin Snack Juice Cookie Cocktail Guacamole Coffee Food Fruit Nightstand Grape Milk Mushroom Pizza Seafood Sushi Tea Taco Strawberry Tomato Wine Cream Bread Lemon Banana Hamburger Orange Peach Coconut Vegetable Cabbage Carrot Mango Pineapple Cake Honeycomb Candy Salad Grapefruit Turkey Doughnut Sandwich Watermelon 'Fast food' 'Hot dog' 'French fries' 'Ice cream' 'Egg (Food)' 'Pizza cutter' 'Submarine sandwich' 'Bell pepper'
``

In [5]:
print(os.getcwd())

/home/jupyter/food-detection/Dataset


In [24]:
from multiprocessing import Pool

image_path = "images"
p = pathlib.Path(image_path)
txt_files = list(p.glob("**/**/*.txt"))
print(len(txt_files))


def check_filter_rules(x):
    invalid_names = ["darknet_obj_names"]
    for name in invalid_names:
        if name in str(x):
            return False
    return True


cleaned_txt_files = list(filter(check_filter_rules, txt_files))
jpg_files = list(p.glob("**/**/*.jpg"))
cleaned_txt_files.extend(jpg_files)
def copyFile(path):
    des = str(path).split('/')[-1]
    des_path = os.path.join(obj_path,des)
    if not os.path.exists(des_path):
        shutil.copy(str(path), obj_path)
        
print(len(jpg_files))
print(len(cleaned_txt_files))

p = Pool(5)
_ = p.map(copyFile,cleaned_txt_files)

59755
59754
119508


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [25]:
!ls obj | wc -l

106902


# Preprocessing UECFOOD256 dataset

In [31]:
print(os.getcwd())

/home/jupyter/food-detection/Dataset


In [32]:
path_folder = 'UECFOOD256'
if not os.path.isdir(path_folder):
    print(f"{path_folder} does not exist!")
    !unzip -qq dataset256.zip
else:
    print(f"{path_folder} exist!")

UECFOOD256 does not exist!


In [33]:
path_root = pathlib.Path(path_folder)
path_root

PosixPath('UECFOOD256')

In [34]:
category_path_txt = os.path.join(path_root, 'category.txt')
category_path_txt

'UECFOOD256/category.txt'

In [35]:
category_df = pd.read_csv(category_path_txt, sep='	', header=0, names=['id', 'name'])
category_df.sample(2)

,id,name
70,71,egg roll
154,155,oatmeal


In [36]:
category_df['name'].replace(' ', '_', regex=True, inplace=True)
category_df['name'].replace('-', '_', regex=True, inplace=True)
category_df.sample(5)

,id,name
191,192,Pork_Sticky_Noodles
213,214,Small_steamed_savory_rice_pancake
179,180,stinky_tofu
254,255,eight_treasure_rice
21,22,soba_noodle


In [37]:
category_df['id'] = category_df['id'].apply(lambda x: os.path.join(path_root, str(x)))
category_df.sample(5)

,id,name
224,UECFOOD256/225,churro
191,UECFOOD256/192,Pork_Sticky_Noodles
227,UECFOOD256/228,ayam_goreng
91,UECFOOD256/92,beef_bowl
141,UECFOOD256/142,ham_cutlet


In [38]:
def generate_annotation(row):
    path = row[0]
    _name = row[1]
    bb_info_df = pd.read_csv(
        os.path.join(path, "bb_info.txt"),
        sep=" ",
        header=0,
        names=["img", "x1", "y1", "x2", "y2"],
    )
    bb_info_df["img_id"] = bb_info_df["img"]
    bb_info_df["img"] = bb_info_df["img"].apply(lambda x: 0)
    bb_info_df.iloc[:, 1:-1] = bb_info_df.iloc[:, 1:-1].astype(float)

    for idx, item in bb_info_df.iterrows():
        img_id = str(int(item[5]))
        name = "UECFOOD256_" + _name + "_" + img_id
        img_id_jpg = img_id + ".jpg"
        img_path = os.path.join(path, img_id_jpg)

        name_txt = name + ".txt"
        name_jpg = name + ".jpg"
        name_jpg_path = os.path.join(path, name_jpg)
        name_path = os.path.join(path, name_txt)
        if not os.path.exists(name_jpg_path):
            os.rename(img_path, name_jpg_path)
            if os.path.exists(name_path):
                # an annotation file already exists for this image so append to it
                open_mode = "+a"
            else:
                # no annotation file exists yet for this image so create it
                open_mode = "+w"
            with open(name_path, open_mode) as darknet_file:
                darknet_file.write(
                    f"{int(item[0])} {item[1]} "
                    f"{item[2]} "
                    f"{item[3]} "
                    f"{item[4]}\n"
                )


for idx, row in category_df.iterrows():
    generate_annotation(row)

In [39]:
p = pathlib.Path(path_folder)
txt_files = list(p.glob('**/*.txt'))
def check_filter_rules(x):
    invalid_names = ['README','category','bb_info']
    for name in invalid_names:
        if name in str(x):
            return False
    return True

cleaned_txt_files = list(filter(check_filter_rules,txt_files))
for path in cleaned_txt_files:
    shutil.move(str(path),obj_path)

jpg_files = list(p.glob('**/*.jpg'))

for path in jpg_files:
    shutil.move(str(path),obj_path)


In [40]:
!ls obj | wc -l

169692


# Preprocessing EgocentricFood 

In [41]:
print(os.getcwd())

/home/jupyter/food-detection/Dataset


In [42]:
path_folder = 'EgocentricFood'
if not os.path.isdir(path_folder):
    print(f"{path_folder} does not exist!")
    !unzip -qq EgocentricFood.zip
else:
    print(f"{path_folder} exist!")

EgocentricFood exist!


In [43]:
food_path = 'EgocentricFood/3'
food_path_text = os.path.join(food_path, "bb_info.txt")
print(food_path)

EgocentricFood/3


In [44]:
bb_info_df = pd.read_csv(
    food_path_text,
    sep=" ",
    header=0,
    names=["im", "x1", "y1", "x2", "y2"],
    dtype={"im": object},
    index_col=False,
)
bb_info_df["img_id"] = bb_info_df["im"]
bb_info_df["im"] = bb_info_df["im"].apply(lambda x: 0)
bb_info_df.iloc[:, 1:-1] = bb_info_df.iloc[:, 1:-1].astype(float)
print(bb_info_df.head(5))
for idx, item in bb_info_df.iterrows():
    img_id = item[5]
    name = "EgocentricFood" + "_" + img_id
    img_id_jpg = img_id + ".jpg"
    img_path = os.path.join(food_path, img_id_jpg)

    name_txt = name + ".txt"
    name_jpg = name + ".jpg"
    name_jpg_path = os.path.join(food_path, name_jpg)
    name_path = os.path.join(food_path, name_txt)
    if not os.path.exists(name_jpg_path):
        os.rename(img_path, name_jpg_path)
        if os.path.exists(name_path):
            # an annotation file already exists for this image so append to it
            open_mode = "+a"
        else:
            # no annotation file exists yet for this image so create it
            open_mode = "+w"
        with open(name_path, open_mode) as darknet_file:
            darknet_file.write(
                f"{int(item[0])} {item[1]} " f"{item[2]} " f"{item[3]} " f"{item[4]}\n"
            )


   im      x1      y1      x2      y2 img_id
0   0     0.0   609.5   552.0  1371.5   0002
1   0    15.5   429.5   483.5  1473.5   0011
2   0    45.5  1107.5   393.5  2013.5   0013
3   0   615.5   753.5   909.5  1083.5   0020
4   0  1017.5  1881.5  1245.5  2169.5   0021


FileNotFoundError: [Errno 2] No such file or directory: 'EgocentricFood/3/0002.jpg' -> 'EgocentricFood/3/EgocentricFood_0002.jpg'

In [45]:
p = pathlib.Path(food_path)
txt_files = list(p.glob('**/*.txt'))
def check_filter_rules(x):
    invalid_names = ['README','category','bb_info']
    for name in invalid_names:
        if name in str(x):
            return False
    return True

cleaned_txt_files = list(filter(check_filter_rules,txt_files))
print(len(cleaned_txt_files))
for path in cleaned_txt_files:
    shutil.move(str(path),obj_path)

jpg_files = list(p.glob('**/*.jpg'))
print(len(jpg_files))
for path in jpg_files:
    shutil.move(str(path),obj_path)

0
0
